In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn.init
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
data_dir = 'C:\\2022_dacon_tourist_ai\\data\\'
df_train = pd.read_csv(data_dir+'train.csv')
df_test = pd.read_csv(data_dir+'test.csv')
df_submit = pd.read_csv(data_dir+'sample_submission.csv')
display(df_train.head())
display(df_test.head())
display(df_submit.head())